In [ ]:
# --- 1. 필수 라이브러리 설치 ---
!pip install -q -U transformers peft accelerate trl datasets huggingface_hub fastapi "uvicorn[standard]" pyngrok bitsandbytes "pandas==2.2.2" "psycopg2-binary"


In [ ]:
# --- 2. 파일 업로드 및 로그인 ---
from google.colab import files
from huggingface_hub import login
import os

print("--- 📂 파일들을 업로드합니다 ---")
files_to_upload = ['gemma-fashion-dpo-final-v3.zip', 'styles.csv']
for filename in files_to_upload:
    if not os.path.exists(filename):
        print(f"\n'{filename}'을 업로드해주세요.")
        files.upload()
    else:
        print(f"'{filename}'이(가) 이미 존재합니다.")


--- 📂 파일들을 업로드합니다 ---
'gemma-fashion-dpo-final-v3.zip'이(가) 이미 존재합니다.
'styles.csv'이(가) 이미 존재합니다.


In [ ]:
# --- 3. 모델 폴더 압축 해제 및 '자동 경로 탐지' ---
print("\n모델 폴더 압축을 해제합니다...")
!unzip -q -o gemma-fashion-dpo-final-v3.zip -d /content/

ADAPTER_PATH_FOUND = ""
for root, dirs, files in os.walk('/content'):
    if 'adapter_config.json' in files:
        ADAPTER_PATH_FOUND = root
        break

if not ADAPTER_PATH_FOUND:
    raise FileNotFoundError("압축 해제 후 'adapter_config.json'을 찾을 수 없습니다. Zip 파일을 확인해주세요.")

print(f"✅ 'adapter_config.json'의 정확한 경로를 찾았습니다: {ADAPTER_PATH_FOUND}")
# ❗ 찾은 경로를 환경 변수에 저장합니다.
os.environ['ADAPTER_PATH'] = ADAPTER_PATH_FOUND



모델 폴더 압축을 해제합니다...
✅ 'adapter_config.json'의 정확한 경로를 찾았습니다: /content/content/gemma-fashion-dpo-final-v3


In [ ]:
#@title 3. API 서버 코드 작성 (serve_final.py) - 최종 폴백 시스템 적용

%%writefile serve_final.py

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import pandas as pd
from itertools import product
import time
import random
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any
import uvicorn
import logging

# --- (기본 설정 및 전역 변수 선언은 이전과 동일) ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
app = FastAPI()
model, tokenizer, styles_info, positive_token_id, negative_token_id = None, None, None, None, None
device = "cuda" if torch.cuda.is_available() else "cpu"

@app.on_event("startup")
def load_resources():
    # ... (이전과 동일한 모델 로딩 로직)
    global model, tokenizer, styles_info, positive_token_id, negative_token_id
    logger.info("✅ 최종 DPO 모델 로딩을 시작합니다...")
    adapter_path = "/content/content/gemma-fashion-dpo-final-v3"
    quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    base_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", quantization_config=quantization_config, device_map="auto")
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    tokenizer.pad_token = tokenizer.eos_token
    positive_token_id = tokenizer.encode("긍정적", add_special_tokens=False)[0]
    negative_token_id = tokenizer.encode("부정적", add_special_tokens=False)[0]
    logger.info("✅ 모델 및 토크나이저 로딩 완료!")
    styles_df = pd.read_csv("styles.csv", on_bad_lines='skip', dtype={'id': int}).set_index('id')
    styles_info = styles_df.to_dict('index')
    logger.info("✅ styles.csv 로드 완료!")

def get_item_description(item_id):
    # ... (이전과 동일)
    if item_id not in styles_info: return ""
    info = styles_info[item_id]; season = info.get('season', 'All')
    return f"{info.get('baseColour', '')} {info.get('articleType', '')} ({season})".strip()

class ClosetItem(BaseModel):
    id: int; gender: str; masterCategory: str; subCategory: str
    articleType: str; baseColor: str; season: str; usage: str


class OutfitRequest(BaseModel):
    closet: List[int]; event: str; temperature: float; condition: str; gender: str

@app.post("/recommend_outfit")
def recommend_outfit(req: OutfitRequest):
    start_time = time.time()

    # --- 1단계: 개별 아이템 평가 (이전과 동일) ---
    temp = req.temperature; event = req.event
    if temp <= 10: suitable_seasons = ['Winter', 'Fall']
    else: suitable_seasons = ['Summer', 'Spring', 'All Season']
    target_usage = ['Casual', 'Smart Casual']
    if event in ["Business Meeting", "Formal Dinner"]: target_usage = ['Formal', 'Smart Casual']
    elif event in ["Gym Workout", "Hiking", "Sports"]: target_usage = ['Sports', 'Active']
    filtered_closet_ids = [ id for id in req.closet if id in styles_info and (styles_info[id].get('season') in suitable_seasons or styles_info[id].get('season') == 'All Season') and (styles_info[id].get('usage') in target_usage) ]
    if not filtered_closet_ids: return {"error": "현재 상황에 맞는 아이템이 옷장에 없습니다."}
    item_prompts = [f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {get_item_description(item_id)}\n결과:" for item_id in filtered_closet_ids]
    inputs = tokenizer(item_prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad(): outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(filtered_closet_ids)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    item_scores = probs[:, 1].tolist()
    scored_items = {filtered_closet_ids[i]: item_scores[i] for i in range(len(filtered_closet_ids))}

    # --- 2단계: 최우수 후보 선정 및 조합 생성 ---
    TOP_K_ITEMS_PER_PART = 3
    classified_items = { 'Topwear': [], 'Bottomwear': [], 'Outerwear': [], 'Footwear': [], 'Full Body': [] }
    outerwear_types = ['Jackets', 'Blazers', 'Waistcoat', 'Coat', 'Shrug', 'Cardigan', 'Nehru Jackets', 'Rain Jacket', 'Sweaters']
    full_body_types = ['Dresses', 'Jumpsuit']
    for item_id in filtered_closet_ids:
        info = styles_info[item_id]; part = None
        if info.get('articleType') in full_body_types: part = 'Full Body'
        elif info.get('articleType') in outerwear_types: part = 'Outerwear'
        elif info.get('subCategory') == 'Topwear': part = 'Topwear'
        elif info.get('subCategory') == 'Bottomwear': part = 'Bottomwear'
        elif info.get('masterCategory') == 'Footwear': part = 'Footwear'
        if part: classified_items[part].append(item_id)

    top_items_by_part = { part: sorted(items, key=lambda x: scored_items.get(x, 0), reverse=True)[:TOP_K_ITEMS_PER_PART] for part, items in classified_items.items() }

    # --- ★ NEW: 지능형 폴백(Fallback) 로직 ---
    def generate_combinations(items_pool):
        """주어진 아이템 풀로 조합을 생성하는 함수"""
        base = list(product(items_pool.get('Topwear', []), items_pool.get('Bottomwear', []), items_pool.get('Footwear', [])))
        outer = list(product(items_pool.get('Topwear', []), items_pool.get('Bottomwear', []), items_pool.get('Outerwear', []), items_pool.get('Footwear', [])))
        full_body = list(product(items_pool.get('Full Body', []), items_pool.get('Footwear', [])))
        return [c for c in base + outer + full_body if c]

    # 1차 시도: 최상위 아이템으로 조합 생성
    all_combinations = generate_combinations(top_items_by_part)

    # 2차 시도: 1차 실패 시, 필터링 통과한 모든 아이템으로 재시도
    if not all_combinations:
        logger.warning("상위 아이템으로 조합 생성 실패. 필터링된 전체 아이템으로 재시도합니다.")
        all_combinations = generate_combinations(classified_items)

    if not all_combinations:
        return {"error": "옷장의 아이템들로 유효한 조합을 만들 수 없습니다."}

    # (이하 AI 최종 평가 및 결과 반환 로직은 이전과 동일)
    prompts = [ f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {', '.join([get_item_description(id) for id in combo])}\n결과:" for combo in all_combinations ]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad(): outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(all_combinations)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    all_scores = probs[:, 1].tolist()
    best_score_index = all_scores.index(max(all_scores))
    best_score = all_scores[best_score_index]
    best_combo = all_combinations[best_score_index]
    best_combo_ids = [int(id_val) for id_val in best_combo if id_val is not None]
    outfit_str = ", ".join([get_item_description(id) for id in best_combo_ids])
    best_outfit_info = {"description": outfit_str, "ids": best_combo_ids}
    explanation = f"긍정적: {req.temperature}°C의 {req.condition} 날씨에 진행되는 {req.event}에 가장 잘 어울리는 스타일입니다."
    end_time = time.time()
    return {"best_combination": best_outfit_info, "explanation": explanation, "best_score": best_score, "processing_time": end_time - start_time}

Overwriting serve_final.py


In [ ]:
# --- 5. 서버 실행 및 ngrok 터널 생성 ---
import threading
import time
import requests
from pyngrok import ngrok
import uvicorn

def run_app():
  uvicorn.run("serve_final:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_app)
thread.start()
print("\n✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...")

print("⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...")
start_time = time.time()
server_ready = False
while time.time() - start_time < 600:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        if response.status_code == 404:
            print("✅ 서버가 응답하기 시작했습니다!")
            server_ready = True
            break
    except requests.exceptions.ConnectionError: time.sleep(5); print("...")
if not server_ready:
    print("❌ 10분이 지나도 서버가 준비되지 않았습니다. 런타임을 재시작하고 다시 시도해주세요.")
else:
    NGROK_TOKEN = "31HHkLEWGt90qDRoAWd6BqiGL9K_5fJsXN7LgijbtcAVwkwAS" # 🚨
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(8000)
    print(f"\n🎉 서버가 성공적으로 생성되었습니다. API 주소: {public_url}")
    print("이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.")

thread.join()


✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...
⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...
...
...
...
...
...


INFO:     Started server process [22481]
INFO:     Waiting for application startup.


✅ 'serve_final.py' 파일이 성공적으로 생성되었습니다.
...
...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...
...
...
...
...


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


...
INFO:     127.0.0.1:42426 - "GET / HTTP/1.1" 404 Not Found
✅ 서버가 응답하기 시작했습니다!

🎉 서버가 성공적으로 생성되었습니다. API 주소: NgrokTunnel: "https://58fea4bf219d.ngrok-free.app" -> "http://localhost:8000"
이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.
INFO:     27.0.238.70:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.70:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     61.42.109.11:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     61.42.109.11:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     27.0.238.187:0 - "GET / HTTP/1.1" 404 Not Found


KeyboardInterrupt: 